In [1]:
import findspark
findspark.init()

In [17]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PWC - Topped Student").master("local[*]").getOrCreate()

24/04/21 23:43:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [18]:
spark

In [19]:
spark.sparkContext.defaultParallelism

4

## Get the Student ID of the student who topped in the respective subjects

In [21]:
data = [(1, 'Math', 90),
        (1, 'Science', 93),
        (1, 'History', 85),
        (2, 'Math', 85),
        (2, 'Science', 79),
        (2, 'History', 96),
        (3, 'Science', 87),
        (3, 'Math', 95),
        (3, 'History', 77),
        (4, 'Math', 78),
        (4, 'Science', 91),
        (4, 'History', 90),
        (5, 'Math', 92),
        (5, 'Science', 84),
        (5, 'History', 88)    
]


schema = ["Std_id", "Subjects", "Marks"]


df = spark.createDataFrame(data=data, schema=schema)

df.show()

+------+--------+-----+
|Std_id|Subjects|Marks|
+------+--------+-----+
|     1|    Math|   90|
|     1| Science|   93|
|     1| History|   85|
|     2|    Math|   85|
|     2| Science|   79|
|     2| History|   96|
|     3| Science|   87|
|     3|    Math|   95|
|     3| History|   77|
|     4|    Math|   78|
|     4| Science|   91|
|     4| History|   90|
|     5|    Math|   92|
|     5| Science|   84|
|     5| History|   88|
+------+--------+-----+



In [28]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc, col

df_rnk = df.withColumn("Rank", rank().over(Window.partitionBy("Subjects").orderBy(desc("Marks"))))

df_rnk.show()

+------+--------+-----+----+
|Std_id|Subjects|Marks|Rank|
+------+--------+-----+----+
|     2| History|   96|   1|
|     4| History|   90|   2|
|     5| History|   88|   3|
|     1| History|   85|   4|
|     3| History|   77|   5|
|     3|    Math|   95|   1|
|     5|    Math|   92|   2|
|     1|    Math|   90|   3|
|     2|    Math|   85|   4|
|     4|    Math|   78|   5|
|     1| Science|   93|   1|
|     4| Science|   91|   2|
|     3| Science|   87|   3|
|     5| Science|   84|   4|
|     2| Science|   79|   5|
+------+--------+-----+----+



In [30]:
df_res = df_rnk.filter(col("Rank") == 1).drop("Rank")

df_res.show()

+------+--------+-----+
|Std_id|Subjects|Marks|
+------+--------+-----+
|     2| History|   96|
|     3|    Math|   95|
|     1| Science|   93|
+------+--------+-----+



In [32]:
spark.stop()